# Items Counters

Analyzes database of games looking for specific items which counter heroes.

In [2]:
import sqlite3
import meta
import os
import ujson as json
import pandas as pd

In [3]:
DB_FILES = ['matches_3_all_2020010307.db']

In [4]:
rows=[]
for db_file in DB_FILES:
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    c.execute("SELECT match_id, radiant_heroes, dire_heroes, items, gold_spent, radiant_win FROM {}".format(
        os.environ['DOTA_SQL_STATS_TABLE']))
    rows.extend(c.fetchall())
    conn.close()
    
print("{0} matches found in database".format(len(rows)))

23149 matches found in database


## Generate histogram of items

Use this to determine short list of items to screen as counters.

In [5]:
item_list=[]
for match_id, radiant_heroes, dire_heroes, items, gold_spent, radiant_win in rows:
    t=json.loads(items)
    for k,v in t.items():
        for item in v:
            if not(item==0):
                item_list.append([match_id, item])

In [6]:
df=pd.DataFrame(item_list)
df.columns=['match_id', 'item']
df['match_id']='dummy'
df_summary=df.pivot_table(values='match_id', columns='item', aggfunc='count').T

In [7]:
df_summary.index=[meta.REVERSE_ITEM[a] for a in df_summary.index]

In [10]:
df_summary.to_csv("temp_item_summary.csv")